<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#Проверка-на-тестовой-выборке" data-toc-modified-id="Проверка-на-тестовой-выборке-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Проверка на тестовой выборке</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Наша задача - обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

**План по выполнению проекта следующий:**

1. Загрузка и подготовка данных.
2. Обучение разных модели. 
3. Выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем необходимые библиотеки:

In [1]:
import numpy as np
import pandas as pd
#import transformers
import re
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#from pymystem3 import Mystem
#from tqdm import notebook

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)

Посмотрим на наши данные:

In [3]:
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159449,And it looks like it was actually you who put ...,0


Посмотрим в каком отношении в датафрейме позитивные и негативные комментарии:

In [4]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Наблюдается дисбаланс классов

Проверим датафрейм на наличие пропусков:

In [5]:
data.isna().sum()

text     0
toxic    0
dtype: int64

Пропусков нет, отлично!

Создадим копию, чтобы не работать с исходными данными:

In [6]:
df = data.copy()

Очистим текст от лишних символов:

In [7]:
def clear_text(text):
    text = text.lower() #приведем к нижнему регистру
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(text.split())

In [8]:
df['text'] = df['text'].apply(clear_text)

Проведем лемматизацию:

In [9]:
#вводим функцию РОS тэг:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

In [10]:
lemmatizer = WordNetLemmatizer()

In [11]:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [12]:
df.head()['text']

0    explanation why the edits made under my userna...
1    d aww he matches this background colour i m se...
2    hey man i m really not trying to edit war it s...
3    more i can t make any real suggestions on impr...
4    you sir are my hero any chance you remember wh...
Name: text, dtype: object

Лемматизация занимает много времени, поэтому пока что проверим работоспособность кода:

In [13]:
df.head()['text'].apply(lemm_text)

0    explanation why the edits make under my userna...
1    d aww he match this background colour i m seem...
2    hey man i m really not try to edit war it s ju...
3    more i can t make any real suggestion on impro...
4    you sir be my hero any chance you remember wha...
Name: text, dtype: object

Лемматизация текста работает, проведем её после разделения выборки!

## Обучение

Разделим наши данные на обучающую (80%), валидационную (10%) и тестовую (10%) выборки.

In [14]:
df, test = train_test_split(
    df, test_size=0.1, random_state=12345, stratify=df['toxic'])

In [15]:
train, valid = train_test_split(
    df, test_size=(1/9), random_state=12345, stratify=df['toxic'])

Проверим правильность разбиения:

In [16]:
train.shape[0], valid.shape[0], test.shape[0]

(127432, 15930, 15930)

In [17]:
train['toxic'].value_counts()

0    114484
1     12948
Name: toxic, dtype: int64

In [18]:
valid['toxic'].value_counts()

0    14311
1     1619
Name: toxic, dtype: int64

In [19]:
test['toxic'].value_counts()

0    14311
1     1619
Name: toxic, dtype: int64

Решим проблему дисбаланса классов для тренировочной выборки:

In [20]:
train_toxic = train[train['toxic']==1]

In [21]:
train_not_toxic = train[train['toxic']==0].sample(38844)

In [22]:
train = train_toxic.append(train_not_toxic)

In [23]:
train['toxic'].value_counts()

0    38844
1    12948
Name: toxic, dtype: int64

Теперь на уменьшенной выборке проведем лемматизацию текста:

In [24]:
%%time
train['text'] = train['text'].apply(lemm_text)

CPU times: user 5min 41s, sys: 27.4 s, total: 6min 8s
Wall time: 6min 9s


In [25]:
%%time
valid['text'] = valid['text'].apply(lemm_text)

CPU times: user 1min 48s, sys: 8.8 s, total: 1min 56s
Wall time: 1min 57s


In [26]:
%%time
test['text'] = test['text'].apply(lemm_text)

CPU times: user 1min 47s, sys: 8.58 s, total: 1min 56s
Wall time: 1min 56s


Предсказать нам нужно столбец `toxic`. Назначим его целевым столбцом.

In [27]:
features_train = train.drop(['toxic'], axis=1)
target_train = train['toxic']

In [28]:
features_valid = valid.drop(['toxic'], axis=1)
target_valid = valid['toxic']

In [29]:
features_test = test.drop(['toxic'], axis=1)
target_test = test['toxic']

Всё верно

Создадим множество из стоп слов:

In [30]:
stopwords = set(nltk_stopwords.words('english'))

Произведем TF-IDF векторизацию:

In [31]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
features_train = count_tf_idf.fit_transform(features_train['text'])

In [32]:
features_valid = count_tf_idf.transform(features_valid['text'])

In [33]:
features_test = count_tf_idf.transform(features_test['text'])

Можем приступать к обучению наших моделей!

### Логистическая регрессия

Первой моделью возьмем регрессию:

In [34]:
regression_model = LogisticRegression(C=5,
                                      random_state=1234,
                                      solver='newton-cg',
                                      max_iter=200,
                                      class_weight='balanced')
regression_model.fit(features_train, target_train)
regression_predict = regression_model.predict(features_valid)

In [35]:
f1_score(target_valid, regression_predict)

0.7230095336253543

Неплохое значение f1-меры, посмотрим на другие модели

### Дерево решений

Второй нашей моделью будет дерево:

In [36]:
tree_model = None
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train) # обучаем модель
    predictions_valid = model.predict(features_valid) # получаем предсказания модели
    result = f1_score(target_valid, predictions_valid) # считаем качество модели
    if result > best_result:
        tree_model = model
        best_result = result
        best_depth = depth
print('f1-мера наилучшей модели на валидационной выборке:', best_result)
print('Глубина дерева =', best_depth)

f1-мера наилучшей модели на валидационной выборке: 0.5947242206235012
Глубина дерева = 10


Значение метрики значительно хуже, чем у регресии

### CatBoost

И последняя модель - бустинг:

In [37]:
%%time
cat_model = CatBoostClassifier(learning_rate=0.3, depth=10, random_state=12345, verbose=False, iterations=200)

cat_model.fit(features_train, target_train)
cat_predict = cat_model.predict(features_valid)

f1_score(target_valid, cat_predict)

CPU times: user 31min 36s, sys: 3.81 s, total: 31min 39s
Wall time: 31min 42s


0.772405297197413

Это лучшее значение метрики!

### Проверка на тестовой выборке

Лучше всего себя показал бустинг. Сделаем проверку на тестовой выборке:

In [39]:
cat_predict_test = cat_model.predict(features_test)
f1_score(target_test, cat_predict_test)

0.7777082028804008

Отличное значение f1-меры, даже лучше чем на валидационной выборке, значит модель не переобучилась.

## Выводы

В данной работе перед нами стояла задача обучить модель, способную классифицировать комментарии на позитивные и негативные.   
Первым делом мы провели обработку текста, очистили его от ненужных символов.   
Далее мы разделили наш датафрейм на тренировочную, валидационную и тестовую выборки, произвели TF-IDF векторизацию.   
Затем мы приступили к обучению наших моделей, ориентируясь на значение метрики - f1-меры.   

Первая модель - *Логистическая регрессия* показала значение f1-меры *0.7230095336253543* на валидационной выборке

Вторая модель - *Дерево решений* показала значение *0.5947242206235012* на валидационной выборке, при глубине дерева 10.  

И третья модель - *CatBoost* показала лучшее значение *0.772405297197413* на валидационной выборке.   

В конечном итоге, нам удалось добиться значаения метрики в **0.7777082028804008** на тестовой выборке. Магазин может смело внедрять нашу модель, она поможет искать токсичные комментарии и отправлять их на модерацию.